# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_Linear import ABSA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_MAMS_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
1,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
2,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, ..."
3,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_OUTPUT = '../../../results/ABSA/MAMS/models/bert_fine_tuned.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/MAMS/models/bert_pre_trained_dropout_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/MAMS/stats/bert_pre_trained_dropout_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, BERT_FINE_TUNED_OUTPUT)
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.2526299953460693
Epoch: 0/2, Batch: 223/2237, Loss: 0.2885449528694153
Epoch: 0/2, Batch: 446/2237, Loss: 0.25791382789611816
Epoch: 0/2, Batch: 669/2237, Loss: 0.24232974648475647
Epoch: 0/2, Batch: 892/2237, Loss: 0.147434264421463
Epoch: 0/2, Batch: 1115/2237, Loss: 0.0882880836725235
Epoch: 0/2, Batch: 1338/2237, Loss: 0.1354515105485916
Epoch: 0/2, Batch: 1561/2237, Loss: 0.08040063083171844
Epoch: 0/2, Batch: 1784/2237, Loss: 0.09888941794633865
Epoch: 0/2, Batch: 2007/2237, Loss: 0.1163133829832077
Epoch: 0/2, Batch: 2230/2237, Loss: 0.19088421761989594
Epoch: 1/2, Batch: 0/2237, Loss: 0.07820041477680206
Epoch: 1/2, Batch: 223/2237, Loss: 0.1257300078868866
Epoch: 1/2, Batch: 446/2237, Loss: 0.11559046804904938
Epoch: 1/2, Batch: 669/2237, Loss: 0.20295719802379608
Epoch: 1/2, Batch: 892/2237, Loss: 0.10481714457273483
Epoch: 1/2, Batch: 1115/2237, Loss: 0.09265143424272537
Epoch: 1/2, Batch: 1338/2237, Loss: 0.2153022736310959
Epoch:

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.973690,0.973690,0.871392,0.973690,0.909867,0.973690,0.889834,462.124252
1,0.973332,0.973332,0.869429,0.973332,0.918997,0.973332,0.892897,463.156325
2,0.972621,0.972621,0.869215,0.972621,0.914541,0.972621,0.890712,463.697592
3,0.967149,0.967149,0.834844,0.967149,0.911839,0.967149,0.870066,457.009927
4,0.973043,0.973043,0.869519,0.973043,0.911786,0.973043,0.889424,457.945434
5,0.973236,0.973236,0.870956,0.973236,0.919035,0.973236,0.893985,463.727643
6,0.965870,0.965870,0.827584,0.965870,0.931433,0.965870,0.874699,466.763255
7,0.971559,0.971559,0.862795,0.971559,0.920731,0.971559,0.889100,465.903500
8,0.971458,0.971458,0.861057,0.971458,0.907090,0.971458,0.881851,470.280651
9,0.973858,0.973858,0.871088,0.973858,0.926488,0.973858,0.897417,474.953734


In [20]:
results.to_csv(STATS_OUTPUT)